# Imports and Setup
To begin, run the cell(s) below to install the necessary setup and imports.

In [ ]:
!sudo apt install python3.8

In [ ]:
!python3.8 --version

In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes

In [ ]:
!conda update -n base -c defaults conda --yes

In [7]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.8/site-packages"))

In [ ]:
import sys
sys.path

In [ ]:
!conda init zsh

In [ ]:
!conda create --name env python=3.8 --yes

In [11]:
!source /usr/local/bin/activate env

In [ ]:
%%bash
pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html
pip install timm==0.4.12
pip install pickle5
pip install scikit-learn
pip install numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/pipline/

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py --help

In [ ]:
!python3.8 /content/drive/MyDrive/Toyota_Smarthome-main/pipline/train.py \
-dataset TSU \
-mode rgb \
-split_setting CS \
-model PDAN \
-num_channel 512 \
-lr 0.0002 \
-kernelsize 3 \
-APtype map \
-epoch 15 \
-batch_size 1 \
-comp_info TSU_CS_RGB_PDAN \
-load_model /content/drive/MyDrive/Toyota_Smarthome-main/model/PDAN_TSU_RGB \
-rgb_model_file P02T01C06

In [ ]:
!pip install ipywidgets

## Initializing Constants

In [ ]:
# INITIALIZING PATH VARIABLES (CHANGE HERE IF DIFFERENT DIRECTORY)

PATH_TO_MODEL_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/model/"
PATH_TO_VIDEO_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/video/"
PATH_TO_FEATURE_FOLDER = "/content/drive/MyDrive/Toyota_Smarthome-main/feature_file/"

# R2

Data Exploration Section:
- Check out the different input videos (with UI)
- See playback of video

In [ ]:
import os
import ipywidgets as widgets

videoList = os.listdir(PATH_TO_VIDEO_FOLDER)

video = widgets.Dropdown(
    options = videoList,
    description='Video:',
    disabled=False,
)
video

In [ ]:
from IPython.display import Video

VIDEO_SELECTED = PATH_TO_VIDEO_FOLDER + video.value
Video(VIDEO_SELECTED)

# R3
Inference Section:
- Load a pretrained model (with UI)
- Choose input video (with UI)
- See inference result in the form out output video with captions that indicte the current detected activity in each video frame

In [ ]:
import os
import ipywidgets as widgets

modelList = os.listdir(PATH_TO_MODEL_FOLDER)

model = widgets.Dropdown(
    options = modelList,
    description='Model:',
    disabled=False,
)

box = widgets.Box(children = [model, video])
box

In [ ]:
#finalized chosen model and video directory

SELECTED_MODEL = PATH_TO_MODEL_FOLDER + model.value
SELECTED_VIDEO = PATH_TO_VIDEO_FOLDER + video.value
print(SELECTED_MODEL)
print(SELECTED_VIDEO)

# FEATURE EXTRACTION

TSU I3D Feature Extraction from video

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/

! git clone https://github.com/v-iashin/video_features.git
! pip install omegaconf==2.0.6 av==8.0.2

In [ ]:
!pwd

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/feature_extract/video_features

In [ ]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(0)

# TODO: RE-ADJUST THE DIRECTORY TO BE TO FEATURE_FILE

# TODO: RE-ADJUST TRAIN_CUSTOM.PY

In [ ]:
# Select the feature type
feature_type = 'i3d'

# Load and patch the config
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/videos/sample_videos/sample_video_1.mp4']
# args.show_pred = True
# args.stack_size = 24
# args.step_size = 24
# args.extraction_fps = 25
args.flow_type = 'raft' # 'pwc' is not supported on Google Colab (cupy version mismatch)
# args.streams = 'flow'

# Load the model
extractor = ExtractI3D(args)

# Extract features
for video_path in args.video_paths:
    # extract name of video from video path (and remove '.mp4' suffix)
    video_name = (video_path.split('/')[-1]).split('.')[0]
    print(video_name)
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    print(feature_dict.items())
    for k, v in feature_dict.items():
      print(k)
      print(v.shape)
      print(v)
      if k == "rgb":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/{video_name}_rgb', v)
      elif k == "flow":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/flow/{video_name}_flow', v)
      elif k == "fps":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/fps/{video_name}_fps', v)
      elif k == "timestamps_ms":
        np.save(f'/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/timestamps_ms/{video_name}_timestamps_ms', v)
      else:
        raise NotImplementedError("Error: Unexpected item in feature_dict!")

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/Toyota_Smarthome-main/pipline/data/extracted_features/rgb/sample_video_1_rgb.npy')
print(data)

## R3.3
See inference results in the form of an output video with captions that indicate the current detected activity in each video frame

**Creating a Caption Model**

In [ ]:
!pip install pafy youtube-dl moviepy
!pip install imageio-ffmpeg
import os
import cv2
import math
import pafy
import imageio
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [4]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!wget -nc --no-check-certificate https://www.crcv.ucf.edu/data/UCF50.rar
#!unrar x UCF50.rar -inul -y

In [ ]:
%cd /content/drive/MyDrive/Toyota_Smarthome-main/video

In [ ]:
# Create a Matplotlib figure
plt.figure(figsize = (30, 30))

# Get Names of all classes in UCF50
all_classes_names = os.listdir('trainVideos')

# Generate a random sample of images each time the cell runs
random_range = random.sample(range(len(all_classes_names)), 7)

# Iterating through all the random samples
for counter, random_index in enumerate(random_range, 1):

    # Getting Class Name using Random Index
    selected_class_Name = all_classes_names[random_index]

    # Getting a list of all the video files present in a Class Directory
    video_files_names_list = os.listdir(f'trainVideos/{selected_class_Name}')

    # Randomly selecting a video file
    selected_video_file_name = random.choice(video_files_names_list)

    # Reading the Video File Using the Video Capture
    video_reader = cv2.VideoCapture(f'trainVideos/{selected_class_Name}/{selected_video_file_name}')
    
    # Reading The First Frame of the Video File
    _, bgr_frame = video_reader.read()

    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()

    # Converting the BGR Frame to RGB Frame 
    rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

    # Adding The Class Name Text on top of the Video Frame.
    cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # Assigning the Frame to a specific position of a subplot
    plt.subplot(5, 4, counter)
    plt.imshow(rgb_frame)
    plt.axis('off')

In [8]:
image_height, image_width = 64, 64
max_images_per_class = 5000 #if less videos in each class, need the change the number down

dataset_directory = "trainVideos"
classes_list = ["Weighlifting","WalkingWithDog","Knitting","PushUps","PlayingViolin","PlayingGuitar","PlayingPiano"]

model_output_size = len(classes_list)

In [9]:
def frames_extraction(video_path):
    # Empty List declared to store video frames
    frames_list = []
    
    # Reading the Video File Using the VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Iterating through Video Frames
    while True:

        # Reading a frame from the video file 
        success, frame = video_reader.read() 

        # If Video frame was not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # Appending the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()

    # returning the frames list 
    return frames_list

In [10]:
def create_dataset():

    # Declaring Empty Lists to store the features and labels values.
    temp_features = [] 
    features = []
    labels = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))

        # Iterating through all the files present in the files list
        for file_name in files_list:

            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)

            # Calling the frame_extraction method for every video file path
            frames = frames_extraction(video_file_path)

            # Appending the frames to a temporary list.
            temp_features.extend(frames)
        
        # Adding randomly selected frames to the features list
        features.extend(random.sample(temp_features, max_images_per_class))

        # Adding Fixed number of labels to the labels list
        labels.extend([class_index] * max_images_per_class)
        
        # Emptying the temp_features list so it can be reused to store all frames of the next class.
        temp_features.clear()

    # Converting the features and labels lists to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels

In [ ]:
features, labels = create_dataset()

In [12]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [13]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)

In [ ]:
# Let's create a function that will construct our model
def create_model():

    # We will use a Sequential model for model construction
    model = Sequential()

    # Defining The Model Architecture
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(model_output_size, activation = 'softmax'))

    # Printing the models summary
    model.summary()

    return model


# Calling the create_model method
model = create_model()

print("Model Created Successfully!")

In [ ]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start Training
model_training_history = model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [16]:
# Creating a useful name for our model, incase you're saving multiple models (OPTIONAL)
model_name = f'captionModel_2Oct.h5'

# Saving your Model
model.save(model_name)

In [17]:
def predict_on_live_video(video_file_path, output_file_path, window_size):

    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        #normalized_frame = resized_frame / 255
        normalized_frame = resized_frame

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:

            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]
          
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)


    #     cv2.imshow('Predicted Frames', frame)

    #     key_pressed = cv2.waitKey(10)

    #     if key_pressed == ord('q'):
    #         break

    # cv2.destroyAllWindows()

    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [22]:
# Setting sthe Window Size which will be used by the Rolling Average Proces
window_size = 1

# Constructing The Output YouTube Video Path
output_video_file_path = f'/content/drive/MyDrive/Toyota_Smarthome-main/video/test_ouput.mp4'
input_video_file_path = f'/content/drive/MyDrive/Toyota_Smarthome-main/video/trainVideos/PlayingViolin/v_PlayingViolin_g01_c01.avi'

# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path, window_size)

# Play Video File in the Notebook

**Using Caption Model Staightway**

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/content/drive/MyDrive/Toyota_Smarthome-main/video/captionModel_new.h5')

In [ ]:
image_height, image_width = 64, 64
classes_list = ["WalkingWithDog","Weighlifting"]
def predict_on_live_video(video_file_path, output_file_path, window_size):

    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:

            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]
          
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)


        # cv2.imshow('Predicted Frames', frame)

        # key_pressed = cv2.waitKey(10)

        # if key_pressed == ord('q'):
        #     break

    # cv2.destroyAllWindows()

    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()
# Setting sthe Window Size which will be used by the Rolling Average Proces
window_size = 20

# Constructing The Output YouTube Video Path
output_video_file_path = f'/content/drive/MyDrive/Toyota_Smarthome-main/video/outputnew3.mp4'
input_video_file_path = f'/content/drive/MyDrive/Toyota_Smarthome-main/video/sample_video_1.mp4'

# Calling the predict_on_live_video method to start the Prediction.
predict_on_live_video(input_video_file_path, output_video_file_path, window_size)


# R4 

- Choose a dataset subfolder to use for training (with UI)
- Initialize a model with a network architecture configured in a separate .py file

In [ ]:
import os
import ipywidgets as widgets

featureFolder = os.listdir(PATH_TO_FEATURE_FOLDER)

feature = widgets.Dropdown(
    options = featureFolder,
    description='Feature:',
    disabled=False,
)
feature


In [ ]:
SELECTED_FEATURE = PATH_TO_FEATURE_FOLDER + feature.value
print(SELECTED_FEATURE)